In [8]:
import cx_Oracle
import pandas as pd

def get_next_sequence_value(target_cursor, sequence_name):
    # Function to get the next value from a sequence
    target_cursor.execute(f"SELECT {sequence_name}.NEXTVAL FROM dual")
    return target_cursor.fetchone()[0]

def val_map_insert(vm_df, target_vm_table, target_cursor, target_conn, sequence_name, map_definition_id):
    try:
        # Remove the first column from vm_df
        vm_df = vm_df.iloc[:, 2:]
        
        # Construct the INSERT INTO statement
        vm_column_names = vm_df.columns.tolist()
        
        insert_vm = f"""
            INSERT INTO {target_vm_table} (
                MAP_ID, MAP_DEFINITION_ID, {', '.join(vm_column_names)}  
            ) VALUES (
                :MAP_ID, :MAP_DEFINITION_ID, :{', :'.join(vm_column_names)} 
            )
        """
        
        for index, row in vm_df.iterrows():
            # Get the next sequence value
            vm_values = {'MAP_ID': get_next_sequence_value(target_cursor, sequence_name)}
            vm_values['MAP_DEFINITION_ID'] = map_definition_id  # Use the provided map_definition_id
            
            # Populate the values dictionary from the DataFrame row
            for col in vm_column_names:
                vm_values[col] = row[col]

            # Execute the INSERT statement with the values dictionary
            target_cursor.execute(insert_vm, vm_values)
            target_conn.commit()  # Commit the transaction
            print(f"Row {index} inserted successfully.")
            
    except cx_Oracle.Error as error:
        print(f"Error executing VM INSERT statement: {error}")


In [9]:
import cx_Oracle
import pandas as pd
import time

# Source database connection
source_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="omsddb.cswg.com:1521/omsd_app"
)

# Target database connection
target_conn = cx_Oracle.connect(
    user="apps",
    password="apps",
    dsn="omsddb.cswg.com:1521/omsd_app"
)


start_time = time.time()
print(f"Start of Task: at {start_time}")

OVERRIDE_FLAG = True
p_map_code = "OTC_ORDER_ENTRY_UI"

# Create cursors for both connections
source_cursor = source_conn.cursor()
target_cursor = target_conn.cursor()

# Select data from the source table
sql_query = f"""SELECT * FROM APPS.XXCMN_VALUE_MAP_DEF WHERE MAP_CODE = '{p_map_code}'"""
source_cursor.execute(sql_query)

# Fetch the data and the column names
def_df = source_cursor.fetchall()

# Retrieve the column names from the cursor description
def_column_names = [desc[0] for desc in source_cursor.description]

# Create a DataFrame and set the column names
def_df = pd.DataFrame(def_df, columns=def_column_names)
print(def_df)

# Now you can reference columns by their names
pn_map_def_id = int(def_df['MAP_DEFINITION_ID'].iloc[0])
print(pn_map_def_id)

vm_query = f"""SELECT *
                 FROM APPS.XXCMN_VALUE_MAP 
                 WHERE MAP_DEFINITION_ID = {pn_map_def_id}
                 ORDER BY map_id"""
source_cursor.execute(vm_query)

# Fetch the data and the column names
vm_df = source_cursor.fetchall()

# Retrieve the column names from the cursor description
vm_column_names = [desc[0] for desc in source_cursor.description]

# Create a DataFrame and set the column names
vm_df = pd.DataFrame(vm_df, columns=vm_column_names)
print(vm_df)

# Close the source cursor and connection
source_cursor.close()
source_conn.close()

# Construct the INSERT INTO statement for DEF

def_df = def_df.iloc[:, 1:]

# Specify the target table name
target_table = "OMS_CMN_DATA.XXCMN_VALUE_MAP_DEF_TEST"

# Construct the INSERT INTO statement
insert_def = f"""
    INSERT INTO {target_table} (
        MAP_DEFINITION_ID, {', '.join(def_column_names[1:])}  
    ) VALUES (
        :MAP_DEFINITION_ID ,:{', :'.join(def_column_names[1:])} 
    )
"""

# Create a cursor for the target database connection
target_cursor = target_conn.cursor()

# Initialize the values dictionary
target_cursor.execute("SELECT OMS_CMN_DATA.XXCMN_VALUE_MAP_DEF_TEST_S.NEXTVAL FROM dual")
next_val = target_cursor.fetchone()[0]
values = {'MAP_DEFINITION_ID': next_val}  # Start with the variable value

# Populate the values dictionary from the DataFrame
for col in def_column_names[1:]:
    values[col] = def_df[col].iloc[0]  # Get values from the DataFrame

# Execute the INSERT statement with the values dictionary
try:
    # Execute your SQL query to check if the MAP_CODE already exists
    target_cursor.execute(f"SELECT  MAP_CODE FROM {target_table} WHERE MAP_CODE = :p_map_code", {"p_map_code": p_map_code})

    # Fetch the results
    result = target_cursor.fetchone()
    print(result)
    
    if result:
        existing_map_code, = result
        print(f"MAP_CODE already exists with Existing MAP_CODE: {existing_map_code}")
    
    else:
        # Execute the INSERT statement with the values dictionary
        target_cursor.execute(insert_def, values)
        target_conn.commit()  # Commit the transaction
        print("INSERT statement executed successfully.")
        print(f"Inserted a new row with MAP_DEFINITION_ID: {next_val}")
        
        # Call the val_map_insert function for value map data
        val_map_insert(vm_df, "OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST", target_cursor, target_conn, "OMS_CMN_DATA.XXCMN_VALUE_MAP_TEST_S", next_val)
except cx_Oracle.Error as error:
    print(f"Error executing DEF INSERT statement: {error}")

# Close the target cursor and connection
target_cursor.close()
target_conn.close()


Start of Task: at 1697193143.0006487
   MAP_DEFINITION_ID            MAP_CODE                           MAP_DESC  \
0                301  OTC_ORDER_ENTRY_UI  OTC Order Entry UI List of Values   

  ACTIVE_DATE_FLAG N_VALUE1_SEQUENCE N_VALUE1_PROMPT_TEXT  \
0                Y              None                 None   

  N_VALUE1_REQUIRED_FLAG N_VALUE1_COLUMN_WIDTH C_VALUE1_SEQUENCE  \
0                   None                  None              None   

  C_VALUE1_PROMPT_TEXT  ... C_VALUE19_UI_SHOW C_VALUE19_UI_UPDATE  \
0             Division  ...              None                None   

  D_VALUE19_UI_SHOW D_VALUE19_UI_UPDATE N_VALUE20_UI_SHOW N_VALUE20_UI_UPDATE  \
0              None                None              None                None   

  C_VALUE20_UI_SHOW C_VALUE20_UI_UPDATE D_VALUE20_UI_SHOW D_VALUE20_UI_UPDATE  
0              None                None              None                None  

[1 rows x 375 columns]
301
   MAP_ID  MAP_DEFINITION_ID N_VALUE1 C_VALUE1 D_VALUE